## MLflow's Model Registry

In [5]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "http://localhost:5001"

In [6]:
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [13]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=10,
    order_by=["metrics.rmse ASC"]
)

In [14]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 8e0f7ac6440b49409ce03a2b25654119, rmse: 6.3436
run id: eaa8e749d4c3444d8f111d0a350fb102, rmse: 6.3436
run id: d6edb4090f8945afa6d156b39186244e, rmse: 6.3436
run id: bb7389e39a0b45f5806fa3c505baacb2, rmse: 6.3436
run id: c9e90f2fb90d4e99b4de8d5bce96b3a9, rmse: 6.3436
run id: 80784e16e86b4fe481560bef6cf5c06e, rmse: 6.3457
run id: f191ba423a2b493192909699aa8f62d0, rmse: 6.3809
run id: bcfcabfbd55841adaaeafa6e1600d5bf, rmse: 6.4557
run id: 9ecdcd6412a04c9fb2e6f4178b52e9c3, rmse: 6.4775
run id: dccbf0c5f6c84d3d92eb739569049832, rmse: 6.4800


In [9]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [16]:
run_id = "8e0f7ac6440b49409ce03a2b25654119"
model_uri = f"runs:/{run_id}/models_mlflow"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi-regressor")

Registered model 'nyc-taxi-regressor' already exists. Creating a new version of this model...
2025/12/28 23:03:14 WARNING mlflow.tracking._model_registry.fluent: Run with id 8e0f7ac6440b49409ce03a2b25654119 has no artifacts at artifact path 'models_mlflow', registering model based on models:/m-35a3dcdb346d48fe86bcfd9fcdd9e6bc instead
2025/12/28 23:03:14 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: nyc-taxi-regressor, version 4
Created version '4' of model 'nyc-taxi-regressor'.


<ModelVersion: aliases=[], creation_timestamp=1766959394350, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1766959394350, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='8e0f7ac6440b49409ce03a2b25654119', run_link='', source='models:/m-35a3dcdb346d48fe86bcfd9fcdd9e6bc', status='READY', status_message=None, tags={}, user_id='', version='4'>

In [17]:
model_name = "nyc-taxi-regressor"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 4, stage: None


/var/folders/40/hqvyh7011wx45kl47mfpbs4w0000gn/T/ipykernel_24657/669935608.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [19]:
model_version = 4
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/var/folders/40/hqvyh7011wx45kl47mfpbs4w0000gn/T/ipykernel_24657/1718116654.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1766959394350, current_stage='Staging', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1766959796435, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='8e0f7ac6440b49409ce03a2b25654119', run_link='', source='models:/m-35a3dcdb346d48fe86bcfd9fcdd9e6bc', status='READY', status_message=None, tags={}, user_id='', version='4'>

In [20]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1766959394350, current_stage='Staging', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='The model version 4 was transitioned to Staging on 2025-12-28', last_updated_timestamp=1766959807804, metrics=None, model_id=None, name='nyc-taxi-regressor', params=None, run_id='8e0f7ac6440b49409ce03a2b25654119', run_link='', source='models:/m-35a3dcdb346d48fe86bcfd9fcdd9e6bc', status='READY', status_message=None, tags={}, user_id='', version='4'>